<a href="https://colab.research.google.com/github/ayush7garg/good-friday-gifts-hackathon/blob/master/good_friday_gifts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Importing Libraries
import pandas as pd
import numpy as np
import io
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt

In [0]:
df = pd.read_csv('train.csv')

In [0]:
df.shape

(20279, 17)

In [0]:
df['volumes'] = df['volumes'].fillna(0)

In [0]:
x_train = df.drop("price",axis=1)
y_train = df["price"]

In [0]:
cor = df.corr()
cor_target = abs(cor["price"])

In [0]:
cor_target

gift_type        0.137145
gift_category    0.286888
gift_cluster     0.282881
lsg_1            0.147712
lsg_2            0.185142
lsg_3            0.006178
lsg_4            0.222104
lsg_5            0.525773
lsg_6            0.019769
is_discounted    0.157463
volumes          0.132591
price            1.000000
Name: price, dtype: float64

In [0]:
cor

,gift_type,gift_category,gift_cluster,lsg_1,lsg_2,lsg_3,lsg_4,lsg_5,lsg_6,is_discounted,volumes,price
gift_type,1.000000,0.075328,-0.123263,0.369368,-0.063898,0.534393,0.055803,-0.073652,0.062589,-0.027933,-0.100975,0.137145
gift_category,0.075328,1.000000,-0.138297,0.003494,-0.020970,-0.069802,0.053803,-0.339733,0.085429,0.011843,-0.232432,0.286888
gift_cluster,-0.123263,-0.138297,1.000000,-0.106383,0.109072,0.067285,0.068613,0.146533,0.025014,0.063864,0.233788,-0.282881
lsg_1,0.369368,0.003494,-0.106383,1.000000,-0.162152,0.524086,-0.126679,-0.125548,-0.033314,-0.193836,0.335478,0.147712
lsg_2,-0.063898,-0.020970,0.109072,-0.162152,1.000000,-0.037239,0.022979,0.040390,-0.039292,0.035532,0.035085,-0.185142
lsg_3,0.534393,-0.069802,0.067285,0.524086,-0.037239,1.000000,0.080235,-0.002776,0.067858,-0.120458,0.303429,0.006178
lsg_4,0.055803,0.053803,0.068613,-0.126679,0.022979,0.080235,1.000000,0.272718,0.371960,0.201302,-0.321000,-0.222104
lsg_5,-0.073652,-0.339733,0.146533,-0.125548,0.040390,-0.002776,0.272718,1.000000,0.163077,0.094139,0.054567,-0.525773
lsg_6,0.062589,0.085429,0.025014,-0.033314,-0.039292,0.067858,0.371960,0.163077,1.000000,0.122089,-0.161461,-0.019769
is_discounted,-0.027933,0.011843,0.063864,-0.193836,0.035532,-0.120458,0.201302,0.094139,0.122089,1.000000,-0.262461,-0.157463


In [0]:
cor.columns

Index(['gift_type', 'gift_category', 'gift_cluster', 'lsg_1', 'lsg_2', 'lsg_3',
       'lsg_4', 'lsg_5', 'lsg_6', 'is_discounted', 'volumes', 'price'],
      dtype='object')

In [0]:
cor_target = pd.DataFrame(cor_target)

In [0]:
cor_target = cor_target.T

In [0]:
to_drop = [i for i in list(cor_target.columns) if cor_target[i]['price']<0.1]

In [0]:
to_drop

['lsg_3', 'lsg_6']

In [0]:
x_train = x_train.drop(columns=to_drop)

In [0]:
x_train.head(15)

,gift_id,gift_type,gift_category,gift_cluster,instock_date,stock_update_date,lsg_1,lsg_2,lsg_4,lsg_5,uk_date1,uk_date2,is_discounted,volumes
0,GF_11156,61,534,3942,2014-02-21 05:07:06.000,2016-11-09 15:49:51.000,3377,5221,1912,10,2014-02-24 08:07:06.000,2014-02-24 07:07:06.000,0,0.0
1,GF_11157,61,534,3942,2014-02-21 06:07:06.000,2016-11-11 13:49:51.000,3377,5221,1912,10,2014-02-22 07:07:06.000,2014-02-24 06:07:06.000,1,0.0
2,GF_15689,584,262,0,2014-02-21 09:30:21.000,2016-03-24 14:46:18.000,5290,1579,1912,9,2016-01-26 00:04:45.000,2016-03-18 02:00:00.000,1,0.0
3,GF_11155,61,534,3942,2014-02-22 05:07:06.000,2016-11-10 16:49:51.000,3377,5221,1912,10,2016-11-07 13:49:51.000,2016-11-06 04:00:00.000,0,0.0
4,GF_11158,61,534,3942,2014-02-22 07:07:06.000,2016-11-10 13:49:51.000,3377,5221,1912,9,2016-11-07 15:49:51.000,2016-11-06 01:00:00.000,1,0.0
5,GF_15686,584,262,0,2014-02-23 07:30:21.000,2016-03-23 15:46:18.000,5290,1579,1912,9,2014-02-25 07:30:21.000,2014-02-24 08:30:21.000,1,0.0
6,GF_15690,584,262,0,2014-02-23 10:30:21.000,2016-03-23 18:46:18.000,5290,1579,1912,9,2016-01-25 02:04:45.000,2016-03-16 02:00:00.000,1,0.0
7,GF_15685,584,262,0,2014-02-24 10:30:21.000,2016-03-24 17:46:18.000,5290,1579,1912,9,2016-01-22 03:04:45.000,2016-03-17 03:00:00.000,0,0.0
8,GF_11159,61,534,3942,2014-02-25 07:07:06.000,2016-11-10 14:49:51.000,3377,5221,1912,10,2016-11-07 16:49:51.000,2015-04-08 12:48:27.025,0,0.0
9,GF_15688,584,262,0,2014-02-25 08:30:21.000,2016-03-24 16:46:18.000,5290,1579,1912,9,2016-01-23 03:04:45.000,2016-03-11 04:00:00.000,1,0.0


In [0]:
x_train = x_train.drop(columns=['instock_date','stock_update_date','uk_date1','uk_date2','gift_id'])

In [0]:
x_train.head(15)

,gift_type,gift_category,gift_cluster,lsg_1,lsg_2,lsg_4,lsg_5,is_discounted,volumes
0,61,534,3942,3377,5221,1912,10,0,0.0
1,61,534,3942,3377,5221,1912,10,1,0.0
2,584,262,0,5290,1579,1912,9,1,0.0
3,61,534,3942,3377,5221,1912,10,0,0.0
4,61,534,3942,3377,5221,1912,9,1,0.0
5,584,262,0,5290,1579,1912,9,1,0.0
6,584,262,0,5290,1579,1912,9,1,0.0
7,584,262,0,5290,1579,1912,9,0,0.0
8,61,534,3942,3377,5221,1912,10,0,0.0
9,584,262,0,5290,1579,1912,9,1,0.0


In [0]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
x_train = sc_X.fit_transform(x_train)

In [0]:
x_train[0:15][:]

array([[-1.74342697,  0.59483247,  0.25133272, -0.71676497,  0.45425411,
         0.47941927,  0.57348482, -0.54599014, -0.64149666],
       [-1.74342697,  0.59483247,  0.25133272, -0.71676497,  0.45425411,
         0.47941927,  0.57348482,  1.8315349 , -0.64149666],
       [-0.39967037, -0.56225996, -1.30001286, -0.00909843, -1.14675209,
         0.47941927,  0.14783155,  1.8315349 , -0.64149666],
       [-1.74342697,  0.59483247,  0.25133272, -0.71676497,  0.45425411,
         0.47941927,  0.57348482, -0.54599014, -0.64149666],
       [-1.74342697,  0.59483247,  0.25133272, -0.71676497,  0.45425411,
         0.47941927,  0.14783155,  1.8315349 , -0.64149666],
       [-0.39967037, -0.56225996, -1.30001286, -0.00909843, -1.14675209,
         0.47941927,  0.14783155,  1.8315349 , -0.64149666],
       [-0.39967037, -0.56225996, -1.30001286, -0.00909843, -1.14675209,
         0.47941927,  0.14783155,  1.8315349 , -0.64149666],
       [-0.39967037, -0.56225996, -1.30001286, -0.00909843, -1

In [0]:
x_train = pd.DataFrame(x_train)

In [0]:
from sklearn.model_selection import train_test_split
x_tr,x_test,y_tr,y_test = train_test_split(x_train,y_train,test_size=0.25,random_state=0)

In [0]:
#SVM model
from sklearn import svm
model = svm.SVR()
model.fit(x_tr,y_tr)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [0]:
y_pred = model.predict(x_test)

In [0]:
from sklearn.metrics import mean_absolute_error
100 - np.sqrt(mean_absolute_error(y_test, y_pred))

91.41400498463125

In [0]:
#KNN model
from sklearn import neighbors
model_knn = neighbors.KNeighborsRegressor(n_neighbors = 3,weights='distance',algorithm='ball_tree',p=1)
model_knn.fit(x_tr, y_tr)

KNeighborsRegressor(algorithm='ball_tree', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=3, p=1,
                    weights='distance')

In [0]:
y_pred_knn = model_knn.predict(x_test)
100 - np.sqrt(mean_absolute_error(y_test, y_pred_knn))

93.70662051060398

In [0]:
#Decision Trees
from sklearn.tree import DecisionTreeRegressor
tree = DecisionTreeRegressor(criterion='mae',max_depth=55000)
tree.fit(x_tr,y_tr)
y_pred_tree=tree.predict(x_test)
100 - np.sqrt(mean_absolute_error(y_test, y_pred_tree))

93.43006061289026

In [0]:
df_test = pd.read_csv('test.csv')

In [0]:
df_test['volumes'] = df_test['volumes'].fillna(0)
x_test1 = df_test.drop(columns=['lsg_3', 'lsg_6','instock_date','stock_update_date','uk_date1','uk_date2','gift_id'])


In [0]:
x_test1.head(15)

In [0]:
x_test1 = sc_X.fit_transform(x_test1)
x_test1 = pd.DataFrame(x_test1)

In [0]:
predictions = model_knn.predict(x_test1)

In [0]:
predictions[0:15]

array([ 38.58685561,  46.02684074,  44.4973813 ,  97.01688266,
        43.67818953, 234.12455094,  83.18400405,  28.8803106 ,
        62.51333333,  62.51333333, 214.79829459,  47.8299361 ,
       177.48420613, 137.18682866, 164.76214408])

In [0]:
from sklearn.linear_model import SGDRegressor
model_sgd=SGDRegressor(loss="epsilon_insensitive",alpha=0.0001,l1_ratio=0.2,penalty='elasticnet',fit_intercept
                   =True,max_iter=2000,learning_rate='invscaling',warm_start=False)

model_sgd.fit(x_tr,y_tr)
y_pred_sgd = model_sgd.predict(x_test)
100 - np.sqrt(mean_absolute_error(y_test, y_pred_sgd))

90.68936147793804

In [0]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators=250,random_state=3,criterion="mae")
regressor.fit(x_tr,y_tr)
y_pred_r = regressor.predict(x_test)


In [0]:
y_pred_r = np.around(y_pred_r,decimals=2)
100 - np.sqrt(mean_absolute_error(y_test, y_pred_r))

94.00664108493304

In [0]:
predictions = regressor.predict(x_test1)
predictions = np.around(predictions,decimals=2)

In [0]:
submission = []
submission.append(df_test["gift_id"])
submission.append(predictions)
submission = np.asarray(submission)
submission = np.transpose(submission)
np.savetxt('prediction.csv',submission,fmt='%s,%f',delimiter=',',header="gift_id,price")

In [0]:
predictions[0:10]

array([ 92.82, 100.94,  62.41,  80.21,  49.99, 191.41,  93.37,  55.02,
       119.48, 119.48])

In [0]:
#Neural Networks aproach
x_neu = df.drop("price",axis=1)
x_neu = x_neu.drop(columns=['instock_date','stock_update_date','uk_date1','uk_date2','gift_id'])
y_neu = df["price"]

In [0]:
x_neu.head(10)

In [0]:
x_neu_a = np.asarray(x_neu)

In [0]:
x_neu_a[0:5]

array([[6.100e+01, 5.340e+02, 3.942e+03, 3.377e+03, 5.221e+03, 5.040e+02,
        1.912e+03, 1.000e+01, 5.540e+02, 0.000e+00, 0.000e+00],
       [6.100e+01, 5.340e+02, 3.942e+03, 3.377e+03, 5.221e+03, 5.040e+02,
        1.912e+03, 1.000e+01, 5.540e+02, 1.000e+00, 0.000e+00],
       [5.840e+02, 2.620e+02, 0.000e+00, 5.290e+03, 1.579e+03, 3.203e+03,
        1.912e+03, 9.000e+00, 1.578e+03, 1.000e+00, 0.000e+00],
       [6.100e+01, 5.340e+02, 3.942e+03, 3.377e+03, 5.221e+03, 5.040e+02,
        1.912e+03, 1.000e+01, 5.540e+02, 0.000e+00, 0.000e+00],
       [6.100e+01, 5.340e+02, 3.942e+03, 3.377e+03, 5.221e+03, 5.040e+02,
        1.912e+03, 9.000e+00, 5.540e+02, 1.000e+00, 0.000e+00]])

In [0]:
x_neu_t = x_neu_a.T

In [0]:
x_neu_t.shape

(11, 20279)

In [0]:
y_neu_t = np.atleast_2d(y_neu)

In [0]:
y_neu_t.shape

(1, 20279)

Initializing parameters for the layers of neural networks

In [0]:
def initialize_parameters_deep(layer_dims):
    """
    Arguments:
    layer_dims -- python array (list) containing the dimensions of each layer in our network
    
    Returns:
    parameters -- python dictionary containing your parameters "W1", "b1", ..., "WL", "bL":
                    Wl -- weight matrix of shape (layer_dims[l], layer_dims[l-1])
                    bl -- bias vector of shape (layer_dims[l], 1)
    """
    
    np.random.seed(3)
    parameters = {}
    L = len(layer_dims)            # number of layers in the network

    for l in range(1, L):
        
        parameters['W' + str(l)] = np.random.randn(layer_dims[l],layer_dims[l-1])*0.01
        parameters['b' + str(l)] = np.zeros([layer_dims[l],1])
        
        
        assert(parameters['W' + str(l)].shape == (layer_dims[l], layer_dims[l-1]))
        assert(parameters['b' + str(l)].shape == (layer_dims[l], 1))

        
    return parameters

Defining the forward propagation steps

In [0]:
def linear_forward(A, W, b):
    """
    Implement the linear part of a layer's forward propagation.

    Arguments:
    A -- activations from previous layer (or input data): (size of previous layer, number of examples)
    W -- weights matrix: numpy array of shape (size of current layer, size of previous layer)
    b -- bias vector, numpy array of shape (size of the current layer, 1)

    Returns:
    Z -- the input of the activation function, also called pre-activation parameter 
    cache -- a python tuple containing "A", "W" and "b" ; stored for computing the backward pass efficiently
    """
    
    Z = np.dot(W,A) + b
    
    assert(Z.shape == (W.shape[0], A.shape[1]))
    cache = (A, W, b)
    
    return Z, cache

In [0]:
def linear_activation_forward(A_prev, W, b, activation):
    """
    Implement the forward propagation for the LINEAR->ACTIVATION layer

    Arguments:
    A_prev -- activations from previous layer (or input data): (size of previous layer, number of examples)
    W -- weights matrix: numpy array of shape (size of current layer, size of previous layer)
    b -- bias vector, numpy array of shape (size of the current layer, 1)
    activation -- the activation to be used in this layer, stored as a text string: "sigmoid" or "relu"

    Returns:
    A -- the output of the activation function, also called the post-activation value 
    cache -- a python tuple containing "linear_cache" and "activation_cache";
             stored for computing the backward pass efficiently
    """
    
    if activation == "sigmoid":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = 1/(1+np.exp(-Z)),Z
    
    elif activation == "relu":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = np.maximum(0,Z),Z
    
    assert (A.shape == (W.shape[0], A_prev.shape[1]))
    cache = (linear_cache, activation_cache)

    return A, cache

In [0]:
def L_model_forward(X, parameters):
    """
    Implement forward propagation for the [LINEAR->RELU]*(L-1)->LINEAR->SIGMOID computation
    
    Arguments:
    X -- data, numpy array of shape (input size, number of examples)
    parameters -- output of initialize_parameters_deep()
    
    Returns:
    AL -- last post-activation value
    caches -- list of caches containing:
                every cache of linear_activation_forward() (there are L-1 of them, indexed from 0 to L-1)
    """

    caches = []
    A = X
    L = len(parameters) // 2                  # number of layers in the neural network
    
    for l in range(1, L):
        A_prev = A 
        A, cache = linear_activation_forward(A_prev, parameters["W"+str(l)], parameters["b"+str(l)], "relu")
        caches.append(cache)
    
    AL, cache = linear_activation_forward(A, parameters["W"+str(L)], parameters["b"+str(L)], "relu")
    caches.append(cache)
    
    assert(AL.shape == (1,X.shape[1]))
            
    return AL, caches

Computing cost

In [0]:
def compute_cost(AL, Y):
    """
    Implement the cost function defined by equation (7).

    Arguments:
    AL -- probability vector corresponding to your label predictions, shape (1, number of examples)
    Y -- true "label" vector (for example: containing 0 if non-cat, 1 if cat), shape (1, number of examples)

    Returns:
    cost -- cross-entropy cost
    """
    
    m = Y.shape[1]

    # Compute loss from aL and y.
    cost = (1/m)*np.sum(abs(AL-Y))
    
    cost = np.squeeze(cost)      # To make sure your cost's shape is what we expect (e.g. this turns [[17]] into 17).
    assert(cost.shape == ())
    
    return cost

Defining Backward Propagation Steps

In [0]:
def linear_backward(dZ, cache):
    """
    Implement the linear portion of backward propagation for a single layer (layer l)

    Arguments:
    dZ -- Gradient of the cost with respect to the linear output (of current layer l)
    cache -- tuple of values (A_prev, W, b) coming from the forward propagation in the current layer

    Returns:
    dA_prev -- Gradient of the cost with respect to the activation (of the previous layer l-1), same shape as A_prev
    dW -- Gradient of the cost with respect to W (current layer l), same shape as W
    db -- Gradient of the cost with respect to b (current layer l), same shape as b
    """
    A_prev, W, b = cache
    m = A_prev.shape[1]

    dW = (1/m)*np.dot(dZ,A_prev.T)
    db = (1/m)*np.sum(dZ,axis=1,keepdims=True)
    dA_prev = np.dot(W.T,dZ)
    
    assert (dA_prev.shape == A_prev.shape)
    assert (dW.shape == W.shape)
    assert (db.shape == b.shape)
    
    return dA_prev, dW, db

In [0]:
def linear_activation_backward(dA, cache, activation):
    """
    Implement the backward propagation for the LINEAR->ACTIVATION layer.
    
    Arguments:
    dA -- post-activation gradient for current layer l 
    cache -- tuple of values (linear_cache, activation_cache) we store for computing backward propagation efficiently
    activation -- the activation to be used in this layer, stored as a text string: "sigmoid" or "relu"
    
    Returns:
    dA_prev -- Gradient of the cost with respect to the activation (of the previous layer l-1), same shape as A_prev
    dW -- Gradient of the cost with respect to W (current layer l), same shape as W
    db -- Gradient of the cost with respect to b (current layer l), same shape as b
    """
    linear_cache, activation_cache = cache
    Z = activation_cache
    gZ = Z
    gZ[gZ>0] = 1
    gZ[gZ<0] = 0

    if activation == "relu":
        dZ = dA*gZ
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
        
    elif activation == "sigmoid":
        dZ = dA*(1/(1+np.exp(-Z)))*(1 - (1/(1+np.exp(-Z))))
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
    
    return dA_prev, dW, db

In [0]:
def L_model_backward(AL, Y, caches):
    """
    Implement the backward propagation for the [LINEAR->RELU] * (L-1) -> LINEAR -> SIGMOID group
    
    Arguments:
    AL -- probability vector, output of the forward propagation (L_model_forward())
    Y -- true "label" vector (containing 0 if non-cat, 1 if cat)
    caches -- list of caches containing:
                every cache of linear_activation_forward() with "relu" (it's caches[l], for l in range(L-1) i.e l = 0...L-2)
                the cache of linear_activation_forward() with "sigmoid" (it's caches[L-1])
    
    Returns:
    grads -- A dictionary with the gradients
             grads["dA" + str(l)] = ... 
             grads["dW" + str(l)] = ...
             grads["db" + str(l)] = ... 
    """
    grads = {}
    L = len(caches) # the number of layers
    m = AL.shape[1]
    Y = Y.reshape(AL.shape) # after this line, Y is the same shape as AL
    
    # Initializing the backpropagation
    dAL = -(np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))
    
    # Lth layer (SIGMOID -> LINEAR) gradients. Inputs: "dAL, current_cache". Outputs: "grads["dAL-1"], grads["dWL"], grads["dbL"]

    current_cache = caches[L-1]
    grads["dA" + str(L-1)], grads["dW" + str(L)], grads["db" + str(L)] = linear_activation_backward(dAL, current_cache, "relu")

    
    # Loop from l=L-2 to l=0
    for l in reversed(range(L-1)):
        # lth layer: (RELU -> LINEAR) gradients.
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = linear_activation_backward(grads["dA"+str(l+1)], current_cache, "relu")
        grads["dA" + str(l)] = dA_prev_temp
        grads["dW" + str(l + 1)] = dW_temp
        grads["db" + str(l + 1)] = db_temp

    return grads

In [0]:
def update_parameters(parameters, grads, learning_rate):
    """
    Update parameters using gradient descent
    
    Arguments:
    parameters -- python dictionary containing your parameters 
    grads -- python dictionary containing your gradients, output of L_model_backward
    
    Returns:
    parameters -- python dictionary containing your updated parameters 
                  parameters["W" + str(l)] = ... 
                  parameters["b" + str(l)] = ...
    """
    
    L = len(parameters) // 2 # number of layers in the neural network


    for l in range(L):
        parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - learning_rate*grads["dW"+str(l+1)]
        parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - learning_rate*grads["db"+str(l+1)]

    return parameters

In [0]:
x_neu_t.shape
y_neu_t.shape

(1, 20279)

In [0]:
layers_dims = [11, 20, 7, 5, 1] #  4-layer model

In [0]:
def L_layer_model(X, Y, layers_dims, learning_rate = 0.0075, num_iterations = 3000, print_cost=False):#lr was 0.009
    """
    Implements a L-layer neural network: [LINEAR->RELU]*(L-1)->LINEAR->SIGMOID.
    
    Arguments:
    X -- data, numpy array of shape (num_px * num_px * 3, number of examples)
    Y -- true "label" vector (containing 0 if cat, 1 if non-cat), of shape (1, number of examples)
    layers_dims -- list containing the input size and each layer size, of length (number of layers + 1).
    learning_rate -- learning rate of the gradient descent update rule
    num_iterations -- number of iterations of the optimization loop
    print_cost -- if True, it prints the cost every 100 steps
    
    Returns:
    parameters -- parameters learnt by the model. They can then be used to predict.
    """

    np.random.seed(1)
    costs = []                         # keep track of cost
    
    # Parameters initialization. (≈ 1 line of code)
    ### START CODE HERE ###
    parameters = initialize_parameters_deep(layers_dims)
    ### END CODE HERE ###
    
    # Loop (gradient descent)
    for i in range(0, num_iterations):

        # Forward propagation: [LINEAR -> RELU]*(L-1) -> LINEAR -> SIGMOID.
        ### START CODE HERE ### (≈ 1 line of code)
        AL, caches = L_model_forward(X, parameters)
        ### END CODE HERE ###
        
        # Compute cost.
        ### START CODE HERE ### (≈ 1 line of code)
        cost = compute_cost(AL, Y)
        ### END CODE HERE ###
    
        # Backward propagation.
        ### START CODE HERE ### (≈ 1 line of code)
        grads = L_model_backward(AL, Y, caches)
        ### END CODE HERE ###
 
        # Update parameters.
        ### START CODE HERE ### (≈ 1 line of code)
        parameters = update_parameters(parameters, grads, learning_rate)
        ### END CODE HERE ###
                
        # Print the cost every 100 training example
        if print_cost and i % 100 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
        if print_cost and i % 100 == 0:
            costs.append(cost)
            
    # plot the cost
    plt.plot(np.squeeze(costs))
    plt.ylabel('cost')
    plt.xlabel('iterations (per hundreds)')
    plt.title("Learning rate =" + str(learning_rate))
    plt.show()
    
    return parameters

In [0]:
parameters = L_layer_model(x_neu_t, y_neu_t, layers_dims, num_iterations = 2500, print_cost = True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in greater
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in less


Cost after iteration 0: nan
Cost after iteration 100: nan


KeyboardInterrupt: ignored